In [22]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import json
import random
from selenium.common.exceptions import NoSuchElementException
import datetime
import re
import csv
global driver
import bash
driver = webdriver.Chrome('C:/Users/tosea/chromedriver.exe')

In [23]:
def get_start_and_end_dates(beginDate, endDate, time_interval="all"):
    dates = []
    dt = datetime.datetime.strptime(beginDate, "%Y%m%d")
    date = beginDate[:]
    while date <= endDate:
        if (int(dt.strftime("%Y"))%4 == 0) and (dt.strftime("%m") == "01") and (int(dt.strftime("%d")) <= time_interval):
            time_interval += 1
            dates.append([date, (datetime.datetime.strptime(date, "%Y%m%d")+datetime.timedelta(time_interval-1)).strftime("%Y%m%d")])
            dt = dt + datetime.timedelta(time_interval)
            date = dt.strftime("%Y%m%d")
            time_interval -= 1
        else:
            dates.append([date, (datetime.datetime.strptime(date, "%Y%m%d")+datetime.timedelta(time_interval-1)).strftime("%Y%m%d")])
            dt = dt + datetime.timedelta(time_interval)
            date = dt.strftime("%Y%m%d")     

    if datetime.datetime.strptime(dates[-1][0], "%Y%m%d") < datetime.datetime.strptime(endDate, "%Y%m%d"):
        date = datetime.datetime.strptime(dates[-1][0], "%Y%m%d")+datetime.timedelta(1)
    dates[-1][1] = endDate
    return dates

In [24]:
def write_csv_title(filename, titles):
    with open(filename, 'w', newline='', encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(titles)

def write_csv_elements(filename, elements):
    with open(filename, 'a+', newline='', encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(elements)


In [25]:
def click_button(button_xpath, url, sleeptime_start, sleeptime_end):
    driver.get(url)
    while True:
        try:
            element = driver.find_element_by_xpath(button_xpath)
            element.click()
            time.sleep(random.randint(sleeptime_start, sleeptime_end))
        except NoSuchElementException:
            break

In [1]:
def scrape_element(url, filename, rounds):
    #press Show More button until the last line
    list2 = []
    driver.get(url)
    # click "show more" button till the end of the page
    while True:
        try:
            element = driver.find_element_by_xpath('//button[text()="Show More"]')
            element.click()
            time.sleep(random.randint(6, 10))
        except NoSuchElementException:
            break

    time.sleep(random.randint(4, 6))
    
    '''
    # Codes below should work for counting times of writing into csv, but not working. This code need further testing
    
    element = driver.find_element_by_xpath('//*[contains(text(),"PRINT EDITION")]')
    length = len(element.text) + 100
    '''
    
    length = rounds

    for i in range(1, length):  #count numbers of articles
            
            while True:
                try:
                    pubdate = driver.find_element_by_xpath(f'//*[@id="site-content"]/div/div[2]/div[2]/ol/li[{i}]/div/time')
                    list2.append(pubdate.text)
                    break
                except NoSuchElementException:
                    list2.append(np.NaN)
                    break
            write_csv_elements(filename, list2)
            
            while True:
                try:
                    title = driver.find_element_by_xpath(f'//*[@id="site-content"]/div/div[2]/div[2]/ol/li[{i}]/div/div/div/a/h4')
                    list2.append(title.text)
                    break
                except NoSuchElementException:
                    list2.append(np.NaN)
                    break
            
            while True:
                try:
                    abstract = driver.find_element_by_xpath(f'//*[@id="site-content"]/div/div[2]/div[2]/ol/li[{i}]/div/div/div/a/p')                  
                    list2.append(abstract.text)
                    break
                except NoSuchElementException:
                    list2.append(np.NaN)
                    break
            
            while True:
                try:
                    link = driver.find_element_by_xpath(f'//*[@id="site-content"]/div/div[2]/div[2]/ol/li[{i}]/div/div/div')
                    el = link.find_element_by_css_selector("a")
                    abc = el.get_attribute("href")
                    list2.append(abc)
                    break
                except NoSuchElementException:
                    list2.append(np.NaN)
                    break
                    
            while True:
                try:
                    author = driver.find_element_by_xpath(f'//*[@id="site-content"]/div/div[2]/div[2]/ol/li[{i}]/div/div/div/a/p[2]')
                    list2.append(author.text)
                    break
                except NoSuchElementException:
                    try:
                        author = driver.find_element_by_xpath(f'//*[@id="site-content"]/div/div[2]/div[2]/ol/li[{i}]/div/div/div/a/p')
                        list2.append(author.text)
                        break
                    except NoSuchElementException:
                        list2.append(np.NaN)
                        break
                    
            while True:
                try:
                    news_desk = driver.find_element_by_xpath(f'//*[@id="site-content"]/div/div[2]/div[2]/ol/li[{i}]/div/div/div/p')
                    list2.append(news_desk.text)
                    break
                except NoSuchElementException:
                    list2.append(np.NaN)
                    break
            '''        
            while True:
                try:
                    linka = driver.find_element_by_xpath(f'//*[@id="site-content"]/div/div[2]/div[2]/ol/li[{i}]/div/div/div/a')
                    driver.find_element_by_link_text(linka.text).click()
                    story = driver.find_element_by_xpath('//*[contains(@id, story)]')
                    list2.append(story.text)
                    break
                except NoSuchElementException:
                    list2.append(np.NaN)
                    break
            driver.execute_script("window.history.go(-1)")
            '''
            
            write_csv_elements(filename, list2)
            list2 = []
            

In [27]:
def final_cut_scrape(q, beginDate, endDate, time_interval, rounds):
    titles = [ "pubdate", "title", "abstract", "link", "author", "news_desk"]   
    
    for i in get_start_and_end_dates(beginDate, endDate, time_interval):
        filename = "nyt"+"_"+q+"_"+str(i[0])+"_"+str(i[1])+"_"+f"Interval{time_interval}"+".csv"
        write_csv_title(filename, titles) 
        url= f"https://www.nytimes.com/search?dropmab=false&endDate={i[1]}&query={q}&sort=oldest&startDate={i[0]}&types=article"
        scrape_element(url, filename, rounds)
        time.sleep(2)


In [28]:
def final_all_scrape(q, beginDate, endDate, rounds):

    titles = [ "pubdate", "title", "abstract", "link", "author", "news_desk"]   
    filename = "nyt"+"_"+q+"_"+str(beginDate)+"_"+str(endDate)+"_"+"alldates"+".csv"
    write_csv_title(filename, titles) 
    url= f"https://www.nytimes.com/search?dropmab=false&endDate={endDate}&query={q}&sort=oldest&startDate={beginDate}&types=article"
    scrape_element(url, filename, rounds)


In [8]:
#1850-1899 all period
'''
final_all_scrape("terrorism", "18500101", "18991231", 1000)
'''

'\nfinal_all_scrape("terrorism", "18500101", "18991231", 1000)\n'

In [9]:
#1900-1919 every 10 years
'''
final_all_scrape("terrorism", "19000101", "19091231", 1000)
final_all_scrape("terrorism", "19100101", "19191231", 1000)
'''

'\nfinal_all_scrape("terrorism", "19000101", "19091231", 1000)\nfinal_all_scrape("terrorism", "19100101", "19191231", 1000)\n'

In [10]:
#1920-1948 every year
'''
for i in range(1920, 1949):
    final_all_scrape("terrorism", f"{i}0101", f"{i}1231", 1000)
'''

'\nfor i in range(1920, 1949):\n    final_all_scrape("terrorism", f"{i}0101", f"{i}1231", 1000)\n'

In [11]:
#1949-1965 every four months
'''
for i in range(1949, 1966):
    final_all_scrape("terrorism", f"{i}0101", f"{i}0430", 1000)
    final_all_scrape("terrorism", f"{i}0501", f"{i}0731", 1000)
    final_all_scrape("terrorism", f"{i}0801", f"{i}1231", 1000)
'''

'\nfor i in range(1949, 1966):\n    final_all_scrape("terrorism", f"{i}0101", f"{i}0430", 1000)\n    final_all_scrape("terrorism", f"{i}0501", f"{i}0731", 1000)\n    final_all_scrape("terrorism", f"{i}0801", f"{i}1231", 1000)\n'

In [12]:
#1966-1969 every three months
'''
for i in range(1966, 1969):
    final_all_scrape("terrorism", f"{i}0101", f"{i}0331", 1000)
    final_all_scrape("terrorism", f"{i}0401", f"{i}0631", 1000)
    final_all_scrape("terrorism", f"{i}0701", f"{i}0930", 1000)
    final_all_scrape("terrorism", f"{i}1001", f"{i}1231", 1000)
'''

'\nfor i in range(1966, 1969):\n    final_all_scrape("terrorism", f"{i}0101", f"{i}0331", 1000)\n    final_all_scrape("terrorism", f"{i}0401", f"{i}0631", 1000)\n    final_all_scrape("terrorism", f"{i}0701", f"{i}0930", 1000)\n    final_all_scrape("terrorism", f"{i}1001", f"{i}1231", 1000)\n'

In [13]:
#1969-2000 every month

for i in range(1995, 2001):
    final_all_scrape("terrorism", f"{i}0101", f"{i}0131", 1000)
    final_all_scrape("terrorism", f"{i}0201", f"{i}0301", 1000)
    final_all_scrape("terrorism", f"{i}0302", f"{i}0331", 1000)
    final_all_scrape("terrorism", f"{i}0401", f"{i}0430", 1000)
    final_all_scrape("terrorism", f"{i}0501", f"{i}0531", 1000)
    final_all_scrape("terrorism", f"{i}0601", f"{i}0630", 1000)
    final_all_scrape("terrorism", f"{i}0701", f"{i}0731", 1000)
    final_all_scrape("terrorism", f"{i}0801", f"{i}0831", 1000)
    final_all_scrape("terrorism", f"{i}0901", f"{i}0930", 1000)
    final_all_scrape("terrorism", f"{i}1001", f"{i}1031", 1000)
    final_all_scrape("terrorism", f"{i}1101", f"{i}1130", 1000)
    final_all_scrape("terrorism", f"{i}1201", f"{i}1231", 1000)



In [14]:
#2001 Jan-Aug

for i in range(2001, 2002):
    final_all_scrape("terrorism", f"{i}0101", f"{i}0131", 1000)
    final_all_scrape("terrorism", f"{i}0201", f"{i}0301", 1000)
    final_all_scrape("terrorism", f"{i}0302", f"{i}0331", 1000)
    final_all_scrape("terrorism", f"{i}0401", f"{i}0430", 1000)
    final_all_scrape("terrorism", f"{i}0501", f"{i}0531", 1000)
    final_all_scrape("terrorism", f"{i}0601", f"{i}0630", 1000)
    final_all_scrape("terrorism", f"{i}0701", f"{i}0731", 1000)
    final_all_scrape("terrorism", f"{i}0801", f"{i}0831", 1000)


In [15]:
#2001 Sep 01-10(10 days)
final_all_scrape("terrorism", "20010901", "20010910", 600)

In [16]:

#2001 Sep 11-30 (Daily)

for i in range(11, 31):
    final_all_scrape("terrorism", f"200109{i}", f"200109{i}", 600)

In [17]:
#2001 Oct-Dec (Every three days)
final_cut_scrape("terrorism", f"20011001", f"20011231", 3, 1000)

In [18]:
#2002-2019 (Every seven days)
final_cut_scrape("terrorism", f"20020101", f"20191231", 7, 1000)

In [29]:
#2002-2019 (Every seven days)
#final_cut_scrape("terrorism", f"20101216", f"20191231", 7, 1000)

In [ ]:
'''
#get links in texts
list1 = []
driver = webdriver.Chrome('C:/Users/tosea/chromedriver.exe')
driver.get(url)

element = driver.find_element_by_xpath('//*[contains(text(),"PRINT EDITION")]')
length = len(element.text) + 1

for i in range(1, length):
    while True:
        try:
            ab = driver.find_element_by_xpath(f'/html/body/div[1]/div[2]/main/div/div[2]/div[2]/ol/li[{i}]/div/div/div/a/p/text()')  
            list1.append(ab.text)
            break
        except NoSuchElementException:
            list1.append(np.NaN)
            break
#First_link = driver.find_element_by_xpath('//*[@id="site-content"]/div/div[2]/div[2]/ol/li/div/div/div')
#el = First_link.find_element_by_css_selector("a")
#abc = el.get_attribute("href")
print("ok")
'''

In [ ]:
#list1

In [ ]:
'''
# get the current page url
el = driver.find_element_by_css_selector("a")
if el:
    url = el.get_attribute("href")
url
'''

In [ ]:
'''
element = driver.find_element_by_xpath('//*[@id="site-content"]/div/div[2]/div[2]/ol/li[2]/div/div/div/a')
print(element.text)
'''

